# Simple Promt in watsonx.ai

In this notebook I will show how to perform  a simple query prompt from Jupyter!

## Step 1- Libraries
First, we load our libraries

In [1]:
from ibm_watson_machine_learning.foundation_models import Model
import os, getpass

## Step 2- Load our Credentials
In order to use the Foundation Models we require load the credentials of IBM Cloud 

### Watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [4]:
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": getpass.getpass("Please enter your WML api key (hit enter): ")
}

Please enter your WML api key (hit enter):  ········


Defining the project id
The API requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

Hint: You can find the project_id as follows. Open the prompt lab in watsonx.ai. At the very top of the UI, there will be Projects / <project name> /. Click on the <project name> link. Then get the project_id from Project's Manage tab (Project -> Manage -> General -> Details).

In [5]:
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

Please enter your project_id (hit enter):  4c0dbedd-56ed-4eea-b36d-02be7323472a


<a id="models"></a>
## Step 3 - Foundation Models on Watsonx

You need to specify `model_id` that will be used for inferencing.

**Action**: Use `FLAN_UL2` model.

In [6]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

In [7]:
model_id = ModelTypes.FLAN_UL2

### Step 4 -Defining the model parameters
We need to provide a set of model parameters that will influence the result:

In [8]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 50
}

Initialize the `Model` class.

In [9]:
#this cell should never fail, and will produce no output
import requests

def getBearer(apikey):
    form = {'apikey': apikey, 'grant_type': "urn:ibm:params:oauth:grant-type:apikey"}
    print("About to create bearer")
#    print(form)
    response = requests.post("https://iam.cloud.ibm.com/oidc/token", data = form)
    if response.status_code != 200:
        print("Bad response code retrieving token")
        raise Exception("Failed to get token, invalid status")
    json = response.json()
    if not json:
        print("Invalid/no JSON retrieving token")
        raise Exception("Failed to get token, invalid response")
    print("Bearer retrieved")
    return json.get("access_token")

In [10]:
credentials["token"] = getBearer(credentials["apikey"])

About to create bearer
Bearer retrieved


In [11]:
model = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id)

### Step 5 - Make the prompt to  `watsonx.ai` model.

In [20]:
def make_prompt(lang, code):
    prompt=f"Respond only with code. Fix this {lang} code: "+ f"{code}"
    return prompt 

In [24]:
prompt_code='''prinf("Hello world)'''

In [25]:
prompt_text=make_prompt("python",prompt_code)

In [26]:
prompt_text

'Respond only with code. Fix this python code: prinf("Hello world)'

### Generate a response
**Note:** Execution of this cell could take few seconds

In [29]:
def generate(prompt_text):
    results = []
    results.append(model.generate_text(prompt=prompt_text))
    return results[0]

In [30]:
generate(prompt_text)

'print("Hello world")'